<a href="https://colab.research.google.com/github/19K41A0518/NNDL/blob/main/NNDL_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment-3**

Build a simple linear regression model using Nesterov Accelerated Gradient (NAG) +SGD optimizer to help 33/11KV substation electric utility to trade power effectively in an hourly ahead energy market by estimating load at a particular hour based on the load at the previous hour. Dataset: Active power load dataset

• Write the python code to build simple linear regression model using NAG+SGD optimizer Do the data normalization 

  • Do the data normalization
 
  • Split the data for train and test (90:10) 
 
  • Train the simple linear regression model training data
 
 • Compute MSE, RMSE and MAE with training data • Compute MSE, RMSE and MAE  with testing data

**Mount Google Drive**

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import libraries**

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

**Read the Data**

In [16]:
data = pd.read_excel('/content/drive/MyDrive/Assignment3.xlsx')
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [17]:
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-1]) 
    y.append(data['Load (kW)'].iloc[i])

In [18]:
data = pd.DataFrame({'Previous Hour Load' : x, 'Present Hour Load' : y})
data.head()

,Previous Hour Load,Present Hour Load
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


**Data Normalisation**

In [19]:
maxX = np.max(data['Previous Hour Load'])
minX = np.min(data['Previous Hour Load'])
maxY = np.max(data['Present Hour Load'])
minY = np.min(data['Present Hour Load'])

data['Previous Hour Load'] = (data['Previous Hour Load'] - minX) / (maxX - minX)
data['Present Hour Load'] = (data['Present Hour Load'] - minY) / (maxY - minY)

**Split the Data**

In [20]:
x = data['Previous Hour Load']
y = data['Present Hour Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=10)

**Nesterov Accelerated Gradient (NAG) +SGD optimizer**

In [21]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1
for _ in range(epochs):   
    for i in range(trainX.shape[0]):
        tempM = m + v*vm
        tempC = c + v*vc       
        gradM = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC)
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        m += vm
        c += vc
print(f'm = {m}\nc = {c}')

m = 0.7794148724880396
c = 0.09669307635526671


In [22]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 465.1832518933884
MSE : 447854.06177873805
RMSE : 669.2189938867083
Testing Error :
MAE : 465.2538263693567
MSE : 414329.73521968076
RMSE : 643.6844997509888


In [23]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,4671.72360,4855.103821
1,5086.98792,5602.420736
2,7441.08768,6049.644532
3,7526.66580,6398.092738
4,3754.68156,4388.683785
...,...,...
1959,4732.51680,4574.449931
1960,5318.15796,6491.158054
1961,5812.76520,6504.401029
1962,4315.53780,4870.412215


In [24]:
x = float(input('Enter the load at previous hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)


Enter the load at previous hour : 5066
Predicted load at present hour : 5221.941354048478
